<img src="../assets/images/Cover.png" alt="Cover" title="AI2E Cover" />

## AI2E - [Workshop 2] - [Data exploration] 

We will go through the essential steps to explore and get the most benefit from our data. 

### Content 
1. Get started with data
2. Feature selection
3. Feature engineering
4. Trait missing values
5. Data visualisation
6. Handle outliers
7. Encode data
8. Scaling
9. Conclusion

### 1. Get started with data:
The dataset provided by an algerian company includes variables about adress of depart, adress of arrival, distance, ... 
    The training dataset provided here is a subset of over 60,000 samples.

#### variables description
<img src="../assets/images/w2_Vdesc.PNG" title="variables description" />


In [1]:
# imports
import pandas as pd
from datetime import date
from matplotlib import pyplot as plt

In [2]:
# read the data
df = pd.read_csv('data/vtc_data.csv')
df.head()

,travel_id,travel_type,car_type,driver_id,address_of_departure,arrival_address,lat_and_long_of_arrival_address,date_of_travel,time_of_travel,estimated_time,distance,options,state,price
0,27550488,advance,Mini Citadine,NaN,"AXA Assurances Algérie, Boulevard du 11 Decemb...","Clinique Krim Belkacem, Boulevard Colonel Krim...","36.770681600000003,3.0510609999999998",2019-09-29 23:55:00,23:55:00,13,6053.0,NaN,cancelled,0.0
1,28026204,live,Mini Citadine,43166.0,Unnamed Road Dar El Beïda,"136 logements, bloc D، Route de Ouled Fayet Ch...","36.745614799999998,2.9428641",2019-09-29 23:54:36,23:54:00,33,31895.0,NaN,finished,133300.0
2,28026119,live,Mini Citadine,30759.0,"Office Riadh El Feth ,bois des arcades ,El Mad...","79 Rue Fabri MARCELLO, Bir Mourad Raïs, 阿尔及利亚","36.736257999999999,3.0616660000000002",2019-09-29 23:54:27,23:54:00,7,3056.0,NaN,finished,32600.0
3,28025871,live,Mini Citadine,32536.0,N24 Bordj El Kiffan,"Dar El Beïda, Algeria","36.727856899999999,3.2181302000000001",2019-09-29 23:47:25,23:47:00,7,6092.0,NaN,finished,38300.0
4,28026102,live,Mini Citadine,40592.0,"Lily Rose, Hydra, Algeria","214 résidence، Bois des Cars 2, Deli Ibrahim, ...","36.753667999999998,2.9762750000000002",2019-09-29 23:46:35,23:46:00,11,6928.0,NaN,finished,47800.0


In [3]:
#get information about integer values
df.describe()

,travel_id,driver_id,estimated_time,distance,price
count,6.533500e+04,60229.000000,65335.000000,65078.000000,6.512100e+04
mean,2.698962e+07,36458.806904,20.731981,12456.091091,4.291896e+04
std,5.911326e+05,5170.877670,21.092258,16331.355123,4.961151e+04
min,1.147333e+07,14770.000000,0.000000,0.000000,-6.800000e+04
25%,2.646277e+07,32376.000000,11.000000,4851.000000,0.000000e+00
50%,2.691974e+07,36638.000000,17.000000,8960.500000,3.820000e+04
75%,2.751333e+07,41457.000000,26.000000,16490.750000,6.080000e+04
max,2.805757e+07,44197.000000,618.000000,575798.000000,2.250000e+06


<b>std: </b>the standard deviation is a measure of the amount of variation or dispersion of a set of values


In [4]:
#get an overall look about the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65335 entries, 0 to 65334
Data columns (total 14 columns):
travel_id                          65335 non-null int64
travel_type                        65335 non-null object
car_type                           65335 non-null object
driver_id                          60229 non-null float64
address_of_departure               65335 non-null object
arrival_address                    65292 non-null object
lat_and_long_of_arrival_address    65292 non-null object
date_of_travel                     65335 non-null object
time_of_travel                     65121 non-null object
estimated_time                     65335 non-null int64
distance                           65078 non-null float64
options                            1152 non-null object
state                              65335 non-null object
price                              65121 non-null float64
dtypes: float64(3), int64(2), object(9)
memory usage: 7.0+ MB


In [ ]:
#check the duration of the whole data
print("First date : ", df["date_of_travel"].min())
print("Last date : ", df["date_of_travel"].max())

### 2. Feature selection
Feature Selection is one of the core concepts in machine learning which hugely impacts the performance of your model. The data features that you use to train your machine learning models have a huge influence on the performance you can achieve.  
<b> Irrelevant or partially relevant features can negatively impact model performance. </b>

In [ ]:
#exo: Define our features and target

In [ ]:
chosen_features = []
target_name = ""

In [ ]:
features = df[chosen_features]
labels = df[target_name]

### 3. Feature engineering
Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work.

In [ ]:
# add hour and day columns

features['date_of_travel'] = pd.DataFrame(pd.to_datetime(features.date_of_travel, format="%Y-%m-%d %H:%M:%S"))

# create a new column
features["hour"] = features["date_of_travel"].dt.hour
features["day_name"] = features["date_of_travel"].dt.day_name()
# drop the "date of travel" column
features.drop(["date_of_travel"], axis = 1, inplace = True)
features.head()

In [ ]:
# separate the lat and lon columns
features[['lat','lon']] = features.lat_and_long_of_arrival_address.str.split(",", expand=True)
features["lat"] = pd.to_numeric(features["lat"], downcast="float")
features["lon"] = pd.to_numeric(features["lon"], downcast="float")
features.drop(["lat_and_long_of_arrival_address"], axis = 1, inplace = True)
features.head()

### 4. Trait missing values
Missing values are one of the most common problems you can encounter when you try to prepare your data for machine learning. The reason for the missing values might be human errors, interruptions in the data flow, privacy concerns, and so on. Whatever is the reason, missing values affect the performance of the machine learning models.

In [ ]:
# check if there are null values in each column
for col in features.columns :
    print(col,':' ,features[col].isnull().sum())

In [ ]:
#trait null values
features["distance"].fillna(features["distance"].mean(), inplace = True)
features["lat"].fillna(features["lat"].mean(), inplace = True)
features["lon"].fillna(features["lon"].mean(), inplace = True)

In [ ]:
# exo: confirm that we don't have null values

### 5. Data visualisation
Data visualization is the graphic representation of data. It involves producing images that communicate relationships among the represented data to viewers of the images.

In [ ]:
# heatmap to show correlation
import seaborn as sns
# we need to use the labels column
corr = pd.concat([features, labels], axis = 1).corr()
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)

<b>correlation: </b> refers to the degree to which a pair of variables are linearly related.

In [ ]:
# print a chart of the estimated time in function of the distance
plt.scatter(features['distance'], labels)
plt.xlabel("distance")
plt.ylabel("estimated time")
plt.show()

### 6. Handle outliers
Before mentioning how outliers can be handled, I want to state that the best way to detect the outliers is to demonstrate the data visually. All other statistical methodologies are open to making mistakes, whereas visualizing the outliers gives a chance to take a decision with high precision.

In [ ]:
# exo: remove all the points which have estimated time > 400

In [ ]:
# exo: re-print the chart to confirm

<b>Note: </b> at this stage and with the informations that we have, we can do features selection again, and that for choosing <b> the best </b> of the features.

### 7. Encode data
One-hot encoding is one of the most common encoding methods in machine learning. This method spreads the values in a column to multiple flag columns and assigns 0 or 1 to them. These binary values express the relationship between grouped and encoded column.

In [ ]:
#Encode data
def oneHotEncode(df, col):
    dfDummies = pd.get_dummies(df[col], prefix = col)
    df = pd.concat([df, dfDummies], axis=1)
    
    # write youe code here
    
    return df

In [ ]:
# exo: complete the oneHotEncode function (drop the current column)

In [ ]:
# applying the function to our data
features = oneHotEncode(features, 'travel_type')
features = oneHotEncode(features, 'car_type')
features = oneHotEncode(features, 'day_name')
features.head()

### 8. Scaling
In most cases, the numerical features of the dataset do not have a certain range and they differ from each other. In real life, it is nonsense to expect age and income columns to have the same range. But from the machine learning point of view, how these two columns can be compared?  
<b> Scaling </b> solves this problem. The continuous features become identical in terms of the range, after a scaling process.

In [ ]:
#scale data
def scale(df, cols):     
    for col in cols:
        
        # write your code here: 
    return df
features = scale(features, ["distance", "hour"]) 
features.head()

In [ ]:
# exo: apply min-max scaling in the previous function

### 9. Conclusion

You are now capable to exploit data and extract the most useful informations from it.

in the next lesson you will learn how to use the our final data to create and train a model.